### Preliminaries

In [ ]:
import wget
#download tiny shakespare
#wget.download("https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt", "../data/")
#wget.download("https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt", "../data/")


In [ ]:
# read it in to inspect it
with open("../data/el_quijote.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [ ]:
print("Lenght of dataset in characters", len(text))

In [ ]:
# let's look at the first characters
print(text[:100])

In [ ]:
# all unique characters from this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars))
print(vocab_size)

In [ ]:
# mapping from chars to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s] # encoder takes a string and returns a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder takes a list of integer and returns a string

print(encode("DON QUIJOTE"))
print(decode(encode("DON QUIJOTE")))

In [ ]:
# enconde the entire text data set to an torch.tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

In [ ]:
# splitting into training and validation set
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When context is {context}, target is {target}")

In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independet sequences will be process in parallel?
block_size = 8 # what is the maximum context lenght for predictions?

def get_batch(split):
    # generate a batch of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print("inputs")
print(xb.shape)
print(xb)
print("targets")
print(yb.shape)
print(yb)


In [ ]:
# biagram language model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BiagramLenguageModel(nn.Module):
    """
    """
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off from the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor integers
        logits = self.token_embedding_table(idx) # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) 
            loss = F.cross_entropy(logits, targets) # measures the quality of logits with respect to the targets

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilitites
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BiagramLenguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

In [ ]:
import numpy as np
import pandas as pd
y = [-(j/100)*np.log(j/100)-(1-j/100)*np.log(1-j/100) for j in range(1, 100, 1)]
x = [j/100 for j in range(1, 100, 1)]
df = pd.DataFrame(dict(
    x = x,
    y = y
))

In [ ]:
yb

In [ ]:
xb

In [ ]:
m.token_embedding_table(xb)

In [ ]:
m.token_embedding_table(torch.tensor([[70]]))

In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(10000):
    # sample a batch
    xb, yb = get_batch("train")

    # evaluate the lost
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

In [ ]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=400)[0].tolist()))

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
5

In [ ]:
!python ../src/biagram.py